# Audio Alignment for Harmonix Set

This notebook tries to align purchased audio with original audio from Harmonix. 

More specifically, for each pair of audio files:
- Load both audio files
- Compute chromagrams
- Use DTW to find the correct start and end points of alignment
- Produce the new aligned mp3s from the purchased audio

In [ ]:
from __future__ import print_function
import glob
import IPython
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

import librosa
from librosa import display
from tqdm import tqdm_notebook as tqdm

# ORIG_MP3_PATH = "/Users/onieto/Desktop/Harmonix/audio/"
# PURC_MP3_PATH = "/Users/onieto/Dropbox/drop/HarmonixMP3_YouTube/"
ORIG_MP3_PATH = "/home/uri/Dropbox/drop/HarmonixMP3_original/"
PURC_MP3_PATH = "/home/uri/Dropbox/drop/HarmonixMP3_YouTube/"
METADATA_TSV = "../dataset/metadata.csv"
OUT_DIR = "aligned_mp3s"
N_FFT = 8192
HOP_SIZE = 1024
METRIC = "euclidean"
SR = 22050
N_MELS = 90

%matplotlib inline

In [ ]:
# Load metadata
meta_df = pd.read_csv(METADATA_TSV, sep=",")
meta_df.head()

In [ ]:
def alignment_score(dtw_curve):
    """The alignment score is simply the average of the difference of 
    the _purchased_ track's DTW alignment curve."""
    return np.mean(np.diff(dtw_curve[:,1][::-1]))

def reconstruct_signal(orig_x, purc_x, dtw_curve):
    """Reconstructs the signal from the purchased signal using the most similar frames
    from the original signal.
    
    We basically take exactly as many frames as the original signal and get the 
    closest to each of these frames from the purchased signal given the dtw curve."""
    orig_dict = {}
    for w in dtw_curve[::-1]:
        orig_dict[w[0]] = w[1]
    y = []
    for i in range(len(orig_dict)):
        samp = orig_dict[i] * HOP_SIZE
        y += list(purc_x[samp:samp + HOP_SIZE])
    last_samp = samp + HOP_SIZE
    y += list(purc_x[last_samp:last_samp + (len(orig_x) - len(y))])
    return y

def compute_alignment(file_id, align_thres=0.9, is_plot=False):
    """Main function to do the alignment between two songs of the same id.
    """
    # Load mp3s
    orig_path = os.path.join(ORIG_MP3_PATH, file_id + ".mp3")
    purc_path = os.path.join(PURC_MP3_PATH, file_id + ".mp3")
    orig_x, _ = librosa.load(orig_path, sr=SR)
    purc_x, _ = librosa.load(purc_path, sr=SR)
    
    # Compute melspecs
    orig_mel = librosa.power_to_db(
        librosa.feature.melspectrogram(y=orig_x, sr=SR, hop_length=HOP_SIZE, n_mels=N_MELS))
    purc_mel = librosa.power_to_db(
        librosa.feature.melspectrogram(y=purc_x, sr=SR, hop_length=HOP_SIZE, n_mels=N_MELS))
    
    # Apply DTW
    D, wp = librosa.sequence.dtw(X=orig_mel, Y=purc_mel, metric='euclidean')
    score = alignment_score(wp)
    
    # Plot
    if is_plot:
        wp_s = np.asarray(wp) * HOP_SIZE / SR
        fig = plt.figure(figsize=(10, 10))
        ax = fig.add_subplot(111)
        librosa.display.specshow(D, x_axis='time', y_axis='time',
                                 cmap='gray_r', hop_length=HOP_SIZE)
        imax = ax.imshow(D, cmap=plt.get_cmap('gray_r'),
                         origin='lower', interpolation='nearest', aspect='auto')
        ax.plot(wp_s[:, 1], wp_s[:, 0], marker='o', color='r')
        plt.title('Warping Path on Acc. Cost Matrix $D$')
        plt.colorbar()
        
    # Return reconstructed signal and score
    return reconstruct_signal(orig_x, purc_x, wp), score

In [ ]:
# Compute alignment for all the dataset, creating new audio files and storing the alignment scores
out = {"File": [], "score": []}
for i, row in tqdm(meta_df.iterrows(), total=len(meta_df)):
    file_id = row["File"]
    
    # Do alignment
    y, score = compute_alignment(file_id)
    
    # Save wav
    librosa.output.write_wav(os.path.join(OUT_DIR, file_id + ".wav"), np.asarray(y), sr=SR)
    
    # Save score
    out["File"].append(file_id)
    out["score"].append(score)

In [ ]:
IPython.display.Audio(data=y, rate=SR)

In [ ]:
out_df = pd.DataFrame(out)
out_df.to_csv("aligned_scores.tsv", sep=",", index=None)